# RPLib Problem 001
## NCAA Men's Basketball Dataset

In [9]:
major_description = "First representative example for the rankability library. Built around the study of NCAA Men's Basketball league."
print(major_description)

First representative example for the rankability library. Built around the study of NCAA Men's Basketball league.


In [10]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import copy
import os
import pandas as pd
import numpy as np
import networkx as nx
from scipy.stats import pearsonr
from scipy.stats import skew
from tqdm import tqdm
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import itertools
import joblib
from scipy import stats

In [12]:
from pathlib import Path
home = str(Path.home())
home

'/home/jupyter-pander14'

In [13]:
import sys
sys.path.insert(0,"%s/rankability_toolbox_dev"%home)
import pyrankability

In [14]:
sys.path.insert(0,"%s/marchmadness_study/"%home)
import base

In [15]:
sys.path.insert(0,"%s/sensitivity_study/src"%home)
import sensitivity_tests
import utilities

ModuleNotFoundError: No module named 'sensitivity_tests'

In [16]:
games={}
remaining_games={}
madness_teams={}
all_teams={}
years = ["2002","2003","2004","2005","2006","2007","2008","2009","2010","2011","2012","2013","2014","2015","2016","2017","2018"]
for year in years:
    games[year],remaining_games[year] = base.read_data(f'{home}/marchmadness_study/data/%steams.txt'%year,f'{home}/marchmadness_study/data/%sgames.txt'%year,f'{home}/marchmadness_study/data/%sMadnessTeams.txt'%year)
    madness_teams[year] = list(np.unique(list(games[year].team1_name.loc[games[year].team1_madness == 1]) + list(games[year].team2_name.loc[games[year].team2_madness == 1])))
    all_teams[year] = list(np.unique(list(games[year].team1_name) + list(games[year].team2_name)))
print(year)
games[year]

2018


,team2,team1,notsure1,date,H_A_N1,points1,H_A_N2,points2,team1_name,team2_name,team1_madness,team2_madness
4147,256,170,737008,2017-11-10,1,92,-1,77,Minnesota,SC_Upstate,0,0
4288,265,293,737008,2017-11-10,1,75,-1,50,Texas_Tech,South_Alabama,1,0
925,56,326,737008,2017-11-10,1,75,-1,60,Villanova,Columbia,1,0
521,32,176,737008,2017-11-10,1,79,-1,78,Monmouth_NJ,Bucknell,0,1
4252,263,235,737008,2017-11-10,1,105,-1,74,Purdue,SIUE,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1985,121,152,737122,2018-03-04,0,65,0,49,Loyola-Chicago,Illinois_St,1,0
933,57,114,737122,2018-03-04,1,81,-1,71,Houston,Connecticut,1,0
1370,81,164,737122,2018-03-04,1,90,-1,70,Memphis,East_Carolina,0,0
1483,88,143,737122,2018-03-04,-1,108,1,96,Lipscomb,FL_Gulf_Coast,1,0


## What does a dataset look like?

In [17]:
remaining_games[year]

,team2,team1,notsure1,date,H_A_N1,points1,H_A_N2,points2,team1_name,team2_name,team1_madness,team2_madness
2,1,73,737130,2018-03-12,1,80,-1,73,Drake,Abilene_Chr,0,0
32,2,313,737125,2018-03-07,1,97,-1,90,UNLV,Air_Force,0,0
42,3,79,737126,2018-03-08,0,67,0,58,E_Michigan,Akron,0,0
59,4,137,737128,2018-03-10,0,86,0,63,Kentucky,Alabama,1,1
68,4,326,737135,2018-03-17,0,81,0,58,Villanova,Alabama,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5478,347,287,737133,2018-03-15,0,73,0,47,Tennessee,Wright_St,1,1
5491,348,197,737126,2018-03-08,0,85,0,75,New_Mexico,Wyoming,0,0
5496,349,92,737136,2018-03-18,0,75,0,70,Florida_St,Xavier,1,1
5498,349,234,737127,2018-03-09,0,75,0,72,Providence,Xavier,1,1


## Running Massey and Colley
Parameters are selected below

In [22]:
direct_thress = [0]
spread_thress = [0]
weight_indirects = [0,0.1,0.2]
domains_ranges = [('all','madness')] # consider all games when constructing D but then only consider march madness teams

In [23]:
games['2002']

,team2,team1,notsure1,date,H_A_N1,points1,H_A_N2,points2,team1_name,team2_name,team1_madness,team2_madness
2237,151,10,731162,2001-11-08,0,71,0,67,Arizona,Maryland,1,1
3937,264,86,731162,2001-11-08,0,72,0,64,Florida,Temple,1,0
1285,86,10,731163,2001-11-09,0,75,0,71,Arizona,Florida,1,1
3940,264,151,731163,2001-11-09,0,82,0,74,Maryland,Temple,1,0
2204,147,261,731166,2001-11-12,1,78,-1,58,Syracuse,Manhattan,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1950,133,8,731277,2002-03-03,0,66,0,58,American_Univ,Lafayette,0,0
2939,198,123,731277,2002-03-03,0,90,0,84,IUPUI,Oakland,0,0
2410,164,128,731277,2002-03-03,-1,95,1,92,Kansas,Missouri,1,1
3704,249,165,731277,2002-03-03,0,70,0,57,Missouri_KC,Southern_Utah,0,0


### Break up the new information into weeks

In [24]:
from datetime import timedelta

days_to_subtracts = [int(d) for d in np.arange(7+28,0,-7)]#[7+28]

delta = timedelta(days=days_to_subtracts[0]) # sample
delta

datetime.timedelta(days=35)

In [25]:
days_to_subtracts

[35, 28, 21, 14, 7]

In [26]:
massey_rankings = {}
colley_rankings = {}
massey_rs = {}
colley_rs = {}
colley_perms = {}
massey_perms = {}

outer_keys = list(itertools.product(domains_ranges,years))
for domain_range,year in tqdm(outer_keys):
    # set the team_domain
    team_domain = None
    if domain_range[0] == 'madness':
        team_domain = madness_teams[year]
    elif domain_range[0] == 'all':
        team_domain = all_teams[year]

    # set the team_range
    team_range = None
    if domain_range[1] == 'madness':
        team_range = madness_teams[year]
    elif domain_range[1] == 'all':
        team_range = all_teams[year]

    columns = ["days_to_subtract","direct_thres","spread_thres","weight_indirect"]+team_range
    massey_rankings[(domain_range,year)] = pd.DataFrame(columns=columns)
    colley_rankings[(domain_range,year)] = pd.DataFrame(columns=columns)
    massey_rs[(domain_range,year)] = pd.DataFrame(columns=columns)
    colley_rs[(domain_range,year)] = pd.DataFrame(columns=columns)
    massey_perms[(domain_range,year)] = pd.DataFrame(columns=columns)
    colley_perms[(domain_range,year)] = pd.DataFrame(columns=columns)

    game_df = pd.DataFrame({"team1_name":games[year]['team1_name'],
                            "team1_score":games[year]['points1'],
                            "team1_H_A_N": games[year]['H_A_N1'],
                            "team2_name":games[year]['team2_name'],
                            "team2_score":games[year]['points2'],
                            "team2_H_A_N": games[year]['H_A_N1'],
                            "date": games[year]['date']
                           }).sort_values(by='date')#.drop('date',axis=1)
    mask = game_df.team1_name.isin(team_domain) & game_df.team2_name.isin(team_domain)
    game_df = game_df.loc[mask]

    keys = list(itertools.product(days_to_subtracts,direct_thress,spread_thress,weight_indirects))

    def compute(days_to_subtract,direct_thres,spread_thres,weight_indirect,team_range,all_teams,game_df,selection_sunday):
        #upper = int(len(game_df)*frac)
        #game_df_sample = game_df.iloc[:upper,:]
        delta = timedelta(days=days_to_subtract)
        game_df_sample = game_df.loc[game_df["date"] <= pd.to_datetime(selection_sunday,format="%m/%d/%Y")-delta].drop('date',axis=1)

        map_func = lambda linked: pyrankability.construct.colley_matrices(linked,direct_thres=direct_thres,spread_thres=spread_thres)
        colley_matrix,colley_b,indirect_colley_matrix,indirect_colley_b = pyrankability.construct.map_vectorized(game_df_sample,map_func)
        colley_matrix = colley_matrix.reindex(index=all_teams,columns=all_teams)
        colley_b = colley_b.reindex(all_teams)
        indirect_colley_matrix = indirect_colley_matrix.reindex(index=all_teams,columns=all_teams)
        indirect_colley_b = indirect_colley_b.reindex(all_teams)
        inxs = []
        for team in team_range:
            inxs.append(int(np.where(colley_b.index == team)[0][0]))
        ranking1,r1,perm1 = pyrankability.rank.ranking_from_matrices(colley_matrix.fillna(0),colley_b.fillna(0),np.array(inxs))
        indirect_ranking1,indirect_r1,indirect_perm1 = pyrankability.rank.ranking_from_matrices(indirect_colley_matrix.fillna(0),indirect_colley_b.fillna(0),np.array(inxs))
        if weight_indirect > 0:
            r1 = r1+weight_indirect*indirect_r1
            perm1, ranking1 = pyrankability.rank.perm_ranking_from_r(r1)
        
        map_func = lambda linked: pyrankability.construct.massey_matrices(linked,direct_thres=direct_thres,spread_thres=spread_thres)
        massey_matrix,massey_b,indirect_massey_matrix,indirect_massey_b = pyrankability.construct.map_vectorized(game_df_sample,map_func)
        massey_matrix = massey_matrix.reindex(index=all_teams,columns=all_teams)
        massey_b = massey_b.reindex(all_teams)
        indirect_massey_matrix = indirect_massey_matrix.reindex(index=all_teams,columns=all_teams)
        indirect_massey_b = indirect_massey_b.reindex(all_teams)
        inxs = []
        for team in team_range:
            inxs.append(int(np.where(massey_b.index == team)[0][0]))
        ranking2,r2,perm2 = pyrankability.rank.ranking_from_matrices(massey_matrix.fillna(0),massey_b.fillna(0),np.array(inxs))
        indirect_ranking2,indirect_r2,indirect_perm2 = pyrankability.rank.ranking_from_matrices(massey_matrix.fillna(0),massey_b.fillna(0),np.array(inxs))
        if weight_indirect > 0:
            r2 = r2+weight_indirect*indirect_r2
            perm2, ranking2 = pyrankability.rank.perm_ranking_from_r(r2)
            
        ranking_values1 = [days_to_subtract,direct_thres,spread_thres,weight_indirect]+list(ranking1)
        ranking_values2 = [days_to_subtract,direct_thres,spread_thres,weight_indirect]+list(ranking2)
        r_values1 = [days_to_subtract,direct_thres,spread_thres,weight_indirect]+list(r1)
        r_values2 = [days_to_subtract,direct_thres,spread_thres,weight_indirect]+list(r2)
        perm_values1 = [days_to_subtract,direct_thres,spread_thres,weight_indirect]+list(perm1)
        perm_values2 = [days_to_subtract,direct_thres,spread_thres,weight_indirect]+list(perm2)
        
        return (pd.Series(ranking_values1,index=columns),
                pd.Series(ranking_values2,index=columns),
                pd.Series(r_values1,index=columns),
                pd.Series(r_values2,index=columns),
                pd.Series(perm_values1,index=columns),
                pd.Series(perm_values2,index=columns)              
               )

    #frac,direct_thres,spread_thres,weight_indirect = keys[0]
    #for frac,direct_thres,spread_thres,weight_indirect in keys:
    #    compute(frac,direct_thres,spread_thres,weight_indirect,team_range,all_teams[year],game_df)
    results = Parallel(n_jobs=-1)(delayed(compute)(days_to_subtract,direct_thres,spread_thres,weight_indirect,team_range,all_teams[year],game_df,base.selectionSundays[year]) for days_to_subtract,direct_thres,spread_thres,weight_indirect in keys)

    c = 0
    for i,key in enumerate(keys):
        days_to_subtract,direct_thres,spread_thres,weight_indirect = key
        colley,massey,colley_r,massey_r,colley_perm,massey_perm = results[i]
        massey.name = c
        colley.name = c
        colley_r.name=c
        massey_r.name=c
        colley_perm.name=c
        massey_perm.name=c
        massey_rankings[(domain_range,year)] = massey_rankings[(domain_range,year)].append(massey)
        colley_rankings[(domain_range,year)] = colley_rankings[(domain_range,year)].append(colley)
        massey_rs[(domain_range,year)] = massey_rs[(domain_range,year)].append(massey_r)
        colley_rs[(domain_range,year)] = colley_rs[(domain_range,year)].append(colley_r)
        massey_perms[(domain_range,year)] = massey_perms[(domain_range,year)].append(massey_perm)
        colley_perms[(domain_range,year)] = colley_perms[(domain_range,year)].append(colley_perm)
        c+=1

100%|██████████| 17/17 [01:32<00:00,  5.50s/it]


## What do we have after running

In [27]:
display(colley_rankings[(domain_range,year)])

,days_to_subtract,direct_thres,spread_thres,weight_indirect,Alabama,Arizona,Arkansas,Auburn,Bucknell,Buffalo,...,Texas_Tech,UMBC,UNC_Greensboro,Villanova,Virginia,Virginia_Tech,West_Virginia,Wichita_St,Wright_St,Xavier
0,35,0,0,0,32,16,34,5,54,41,...,13,63,50,2,1,44,23,22,51,4
1,35.0,0.0,0.0,0.1,32.0,16.0,34.0,5.0,53.0,41.0,...,13.0,63.0,50.0,2.0,1.0,44.0,23.0,22.0,52.0,3.0
2,35.0,0.0,0.0,0.2,32.0,16.0,34.0,5.0,53.0,41.0,...,13.0,63.0,50.0,2.0,1.0,44.0,23.0,22.0,52.0,3.0
3,28,0,0,0,34,16,27,4,58,44,...,11,62,50,2,1,37,22,17,52,3
4,28.0,0.0,0.0,0.1,34.0,16.0,28.0,4.0,58.0,44.0,...,10.0,62.0,50.0,2.0,1.0,37.0,22.0,17.0,52.0,3.0
5,28.0,0.0,0.0,0.2,34.0,16.0,28.0,4.0,58.0,44.0,...,10.0,62.0,50.0,2.0,1.0,37.0,22.0,17.0,52.0,3.0
6,21,0,0,0,31,16,21,5,56,45,...,10,61,49,2,1,38,23,13,54,3
7,21.0,0.0,0.0,0.1,31.0,16.0,21.0,5.0,56.0,45.0,...,10.0,61.0,49.0,2.0,1.0,38.0,23.0,13.0,54.0,3.0
8,21.0,0.0,0.0,0.2,31.0,16.0,21.0,5.0,56.0,45.0,...,10.0,61.0,49.0,2.0,1.0,38.0,23.0,13.0,54.0,3.0
9,14,0,0,0,40,17,24,7,55,46,...,19,63,49,3,1,37,22,12,56,2


## Select your parameters

In [43]:
inx_values = ['domain','range','direct_thres','spread_thres','weight_indirect']
sel_index = ('all','madness',0,0,0.1)
sel_index

('all', 'madness', 0, 0, 0.1)

One way to focus our analysis is to study the parameters that result in the best accuracy when predicting future games. Let's see how the parameters you picked perform on predicting march madness

In [36]:
def calc_predictability(games,r):
    numberCorrectPredictions = 0
    numGames = 0
    for i in games.index:
        team1ID = games.loc[i, "team1_name"]
        team1Score = games.loc[i, "points1"]
        team2ID = games.loc[i, "team2_name"]
        team2Score = games.loc[i, "points2"]
        
        if team1ID in r.index and team2ID in r.index:
            if team1Score > team2Score and r.loc[team1ID] > r.loc[team2ID]:
                numberCorrectPredictions += 1
            elif team2Score > team1Score and r.loc[team2ID] > r.loc[team1ID]:
                numberCorrectPredictions += 1
            elif team1Score == team2Score and r.loc[team1ID] == r.loc[team2ID]:
                numberCorrectPredictions += 1

            numGames += 1
           
    predictability = numberCorrectPredictions/numGames*100
    return predictability,numGames

In [37]:
pred_df = pd.DataFrame(columns=['days_to_subtract','domain','range',"direct_thres","spread_thres","weight_indirect",'Method','Year','Predictability','rankings'])

keys = list(itertools.product(days_to_subtracts,domains_ranges,direct_thress,spread_thress,weight_indirects,years))

c=0
for days_to_subtract,domain_range,dt,st,iw,year in tqdm(keys):
    dom = domain_range[0]
    ran = domain_range[1]
    method = 'Massey'
    r = massey_rs[(domain_range,year)].set_index(["days_to_subtract","direct_thres","spread_thres","weight_indirect"]).loc[(days_to_subtract,dt,st,iw)]
    rankings = massey_rankings[(domain_range,year)].set_index(["days_to_subtract","direct_thres","spread_thres","weight_indirect"]).loc[(days_to_subtract,dt,st,iw)]
    val = calc_predictability(remaining_games[year],r)[0]
    entry = pd.Series([days_to_subtract,dom,ran,dt,st,iw,method,year,val,rankings],name=c,index=pred_df.columns)
    c+=1
    pred_df=pred_df.append(entry)

    method = 'Colley'
    r = colley_rs[(domain_range,year)].set_index(["days_to_subtract","direct_thres","spread_thres","weight_indirect"]).loc[(days_to_subtract,dt,st,iw)]
    rankings = colley_rankings[(domain_range,year)].set_index(["days_to_subtract","direct_thres","spread_thres","weight_indirect"]).loc[(days_to_subtract,dt,st,iw)]
    val = calc_predictability(remaining_games[year],r)[0]
    entry = pd.Series([days_to_subtract,dom,ran,dt,st,iw,method,year,val,rankings],name=c,index=pred_df.columns)
    c+=1
    pred_df=pred_df.append(entry)
    

100%|██████████| 255/255 [00:10<00:00, 22.76it/s]


In [40]:
scores = pred_df.groupby(['Method','days_to_subtract','domain','range','direct_thres','spread_thres','weight_indirect'])['Predictability'].median().reset_index()
scores.sort_values(by='Predictability').set_index('Method').loc['Colley']

,days_to_subtract,domain,range,direct_thres,spread_thres,weight_indirect,Predictability
Method,,,,,,,
Colley,21,all,madness,0,0,0.2,64.044944
Colley,21,all,madness,0,0,0.1,64.044944
Colley,21,all,madness,0,0,0.0,64.044944
Colley,35,all,madness,0,0,0.1,64.285714
Colley,35,all,madness,0,0,0.0,64.285714
Colley,7,all,madness,0,0,0.0,64.285714
Colley,14,all,madness,0,0,0.1,64.444444
Colley,14,all,madness,0,0,0.0,64.444444
Colley,28,all,madness,0,0,0.0,64.444444


In [47]:
best_df = scores.groupby(['days_to_subtract','Method']).apply(lambda df: pd.Series(sel_index,index=inx_values)).reset_index()
best_df

,days_to_subtract,Method,domain,range,direct_thres,spread_thres,weight_indirect
0,7,Colley,all,madness,0,0,0.1
1,7,Massey,all,madness,0,0,0.1
2,14,Colley,all,madness,0,0,0.1
3,14,Massey,all,madness,0,0,0.1
4,21,Colley,all,madness,0,0,0.1
5,21,Massey,all,madness,0,0,0.1
6,28,Colley,all,madness,0,0,0.1
7,28,Massey,all,madness,0,0,0.1
8,35,Colley,all,madness,0,0,0.1
9,35,Massey,all,madness,0,0,0.1


In [51]:
graph_df

,domain,range,direct_thres,spread_thres,weight_indirect,days_to_subtract,Method,Predictability
0,all,madness,0,0,0.1,7,Colley,64.444444
1,all,madness,0,0,0.1,7,Massey,65.306122
2,all,madness,0,0,0.1,14,Colley,64.444444
3,all,madness,0,0,0.1,14,Massey,65.979381
4,all,madness,0,0,0.1,21,Colley,64.044944
5,all,madness,0,0,0.1,21,Massey,66.279070
6,all,madness,0,0,0.1,28,Colley,64.444444
7,all,madness,0,0,0.1,28,Massey,64.516129
8,all,madness,0,0,0.1,35,Colley,64.285714
9,all,madness,0,0,0.1,35,Massey,65.476190


In [48]:
import altair as alt

graph_df = best_df.set_index(inx_values+["days_to_subtract","Method"]).join(scores.set_index(inx_values+["days_to_subtract","Method"])).reset_index().sort_values(by='days_to_subtract')

g = alt.Chart(graph_df).mark_line().encode(
    x='days_to_subtract',
    y=alt.Y('Predictability',scale=alt.Scale(domain=[60, 70])),
    color='Method:N'
)

## How does predictability look?

In [49]:
g

alt.Chart(...)

In [69]:
top_k = 10
feature_names = [f'top{top_k}_jaccard',f'top{top_k}_tau']
ms = pd.DataFrame(columns=['days_to_subtract1','days_to_subtract2','domain','range',"direct_thres","spread_thres","weight_indirect",'Method','Year','rankings1','rankings2']+feature_names)

#keys = list(itertools.product(domains_ranges,direct_thress,spread_thress,weight_indirects,years))
pair_days_to_subtracts = [sorted(days_to_subtracts) for days_to_subtracts in list(itertools.combinations(days_to_subtracts,2))]

c=0
for index,row in best_df.iterrows():
    dom,ran,dt,st,iw = row.loc['domain'],row.loc['range'],row.loc['direct_thres'],row.loc['spread_thres'],row.loc['weight_indirect']
    method = row.loc['Method']
    for year in years:
        for days_to_subtract1,days_to_subtract2 in pair_days_to_subtracts:
            if method == 'Massey':
                rankings = massey_rankings[(domain_range,year)].set_index(["direct_thres","spread_thres","weight_indirect"]).loc[(dt,st,iw)]
            elif method == 'Colley':
                rankings = colley_rankings[(domain_range,year)].set_index(["direct_thres","spread_thres","weight_indirect"]).loc[(dt,st,iw)]
                #import pdb; pdb.set_trace()
            else:
                raise Exception('Unsupported')
            rankings1 = rankings.set_index('days_to_subtract').loc[days_to_subtract1].T
            rankings1 = rankings1.loc[rankings1 < top_k]
            rankings2 = rankings.set_index('days_to_subtract').loc[days_to_subtract2].T
            rankings2_rankings1 = rankings2.loc[rankings1.index]
            rankings2 = rankings2.loc[rankings2 < top_k]
            val = len(set(rankings1.index).intersection(set(rankings2.index)))/len(set(rankings1.index).union(set(rankings2.index)))#top_k
            tau, p_value = stats.kendalltau(np.argsort(rankings1.values), np.argsort(rankings2.values))
            entry = pd.Series([days_to_subtract1,days_to_subtract2,dom,ran,dt,st,iw,method,year,rankings1,rankings2,val,tau],name=c,index=ms.columns)
            c+=1
            ms=ms.append(entry)

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:15: PerformanceWarning: indexing past lexsort depth may impact performance.
  from ipykernel import kernelapp as app


In [70]:
ms['days_diff'] = ms.days_to_subtract2 - ms.days_to_subtract1
ms.head()

,days_to_subtract1,days_to_subtract2,domain,range,direct_thres,spread_thres,weight_indirect,Method,Year,rankings1,rankings2,top10_jaccard,top10_tau,days_diff
0,28,35,all,madness,0,0,0.1,Colley,2002,Alabama 6.0 Arizona 7.0 Cincinnati...,Alabama 6.0 Arizona 7.0 Cincinnati...,0.800000,0.388889,7
1,21,35,all,madness,0,0,0.1,Colley,2002,Alabama 5.0 Arizona 9.0 Cincinnati...,Alabama 6.0 Arizona 7.0 Cincinnati...,0.636364,-0.111111,14
2,14,35,all,madness,0,0,0.1,Colley,2002,Alabama 6.0 Arizona 9.0 Cincinnati...,Alabama 6.0 Arizona 7.0 Cincinnati...,0.800000,-0.055556,21
3,7,35,all,madness,0,0,0.1,Colley,2002,Alabama 6.0 Arizona 7.0 Cincinnati...,Alabama 6.0 Arizona 7.0 Cincinnati...,0.800000,0.000000,28
4,21,28,all,madness,0,0,0.1,Colley,2002,Alabama 5.0 Arizona 9.0 Cincinnati...,Alabama 6.0 Arizona 7.0 Cincinnati...,0.800000,-0.722222,7


In [71]:
import altair as alt

alt.Chart(ms).mark_bar().encode(
    alt.X("top10_jaccard:Q", bin=True),
    y='count()',
    row='days_diff'
)


alt.Chart(...)

In [72]:
import altair as alt

alt.Chart(ms).mark_bar().encode(
    alt.X("top10_tau:Q", bin=True),
    y='count()',
    row='days_diff'
)

alt.Chart(...)

In [73]:
sensitivity_target = ms.copy()

sensitivity_data = {}
for year in tqdm(years):
    sensitivity_data[year] = {}
    for days_to_subtract in days_to_subtracts:
        game_df = pd.DataFrame({"team1_name":games[year]['team1_name'],
                                "team1_score":games[year]['points1'],
                                "team1_H_A_N": games[year]['H_A_N1'],
                                "team2_name":games[year]['team2_name'],
                                "team2_score":games[year]['points2'],
                                "team2_H_A_N": games[year]['H_A_N1'],
                                "date": games[year]['date']
                               }).sort_values(by='date')
        mask = game_df.team1_name.isin(team_domain) & game_df.team2_name.isin(team_domain)
        game_df = game_df.loc[mask]
        delta = timedelta(days=days_to_subtract)
        game_df_sample = game_df.loc[game_df["date"] <= pd.to_datetime(base.selectionSundays[year],format="%m/%d/%Y")-delta].drop('date',axis=1)
        sensitivity_data[year][f"days_to_subtract={days_to_subtract}"]=game_df_sample
        
joblib.dump({'description':major_description,'target':sensitivity_target,'data':sensitivity_data,'other':{'madness_teams':madness_teams,'remaining_games':remaining_games,'best_df':best_df,'top_k':top_k,'feature_names':feature_names}},"generate.joblib.z")

100%|██████████| 17/17 [00:00<00:00, 52.47it/s]


['generate.joblib.z']